In [1]:
import os,sys
#sys.path.append('/work/qdiff/mo_utils')

In [2]:
from mo_utils.utils.tmux_utils import get_session_list,tmux_session,get_session_name,kill_session
from pathlib import Path

libtmux not installed ??


In [3]:
get_session_list()

[Session($3 jupyer),
 Session($0 sd_quantize_wb=8_wsym=True_gpu_0),
 Session($1 sd_quantize_wb=8_wsym=True_gpu_1),
 Session($2 sd_quantize_wb=8_wsym=True_gpu_3)]

In [4]:
#kill_session()

In [5]:
#w8bit_sym,nbit,symmetric = '/fastdata/users/nadavg/sd/qdiff/output_quantization/2025-01-26-18-44-03/ckpt.pth',8,True

In [6]:
nbit,symmetric = 8,True

In [25]:
task = 'quantize'
gpu = 1
prompt = "a puppy wearing a hat" 
weight_bit = nbit
symmetric_weight = symmetric#True
bs = 8
act_bit = 16
outdir= "/workspace/sd/qdiff/output_quantization"
quant_act_ops = True #False #True#True
split_to_16bits = False

resume_w = False
accum_batches = True#True
quantized_ckpt_path = '/home/nadavg/q-diffusion/output_quantization/2025-01-22-16-40-17/wc_ckpt.pth'
#quantized_ckpt_path = w8bit_sym
ddim_steps = 20


if ddim_steps == 20:
    cali_n= 128
    cali_st = 10
    cali_data_path = "/fastdata/users/nadavg/sd/qdiff/gen_calib/calib_dict_steps20.pt"
    cali_iters = 20000#5000 
    cali_iters_a = 5000
    #act_bit = 16
elif ddim_steps == 50:
    cali_n= 128
    cali_st = 25
    cali_data_path = "/fastdata/users/nadavg/sd/qdiff/gen_calib/calib_dict_steps50.pt"
    cali_iters = 20000#5000 
    cali_iters_a = 5000 
else:
    raise ValueError('ddim_steps must be 20 or 50')

#cali_data_path='/fastdata/users/nadavg/sd/qdiff/sd_coco-s75_sample1024_allst.pt'


debug = False#True#False


In [26]:
cmd=(f"python scripts/txt2img.py --prompt '{prompt}' --plms --cond --ptq --weight_bit {weight_bit} --quant_mode qdiff "+
    f"--quant_act --act_bit {act_bit} --cali_st {cali_st} --cali_batch_size {bs} --cali_n {cali_n} --no_grad_ckpt --split --running_stat "+
    f"--sm_abit 16 --cali_data_path {cali_data_path} --outdir {outdir} --ddim_steps {ddim_steps}" +
    symmetric_weight*" --symmetric_weight "+
    resume_w*f"--resume_w --cali_ckpt {quantized_ckpt_path} "+
    quant_act_ops*" --quant_act_ops "+
    split_to_16bits*" --split_to_16bits "+
    accum_batches*" --accum_batches "+
    f"--cali_iters {cali_iters} --cali_iters_a {cali_iters_a} "+
    debug*" --debug "
    )


In [27]:
cmd

"python scripts/txt2img.py --prompt 'a puppy wearing a hat' --plms --cond --ptq --weight_bit 8 --quant_mode qdiff --quant_act --act_bit 16 --cali_st 10 --cali_batch_size 8 --cali_n 128 --no_grad_ckpt --split --running_stat --sm_abit 16 --cali_data_path /fastdata/users/nadavg/sd/qdiff/gen_calib/calib_dict_steps20.pt --outdir /workspace/sd/qdiff/output_quantization --ddim_steps 20 --symmetric_weight  --quant_act_ops  --accum_batches --cali_iters 20000 --cali_iters_a 5000 "

In [28]:
inst_list = [f'cd {Path.home() / "q-diffusion"}',
             f'conda activate qdiff',
             f'export CUDA_VISIBLE_DEVICES={gpu}',
             cmd]

In [29]:
inst_list

['cd /home/nadavg/q-diffusion',
 'conda activate qdiff',
 'export CUDA_VISIBLE_DEVICES=1',
 "python scripts/txt2img.py --prompt 'a puppy wearing a hat' --plms --cond --ptq --weight_bit 8 --quant_mode qdiff --quant_act --act_bit 16 --cali_st 10 --cali_batch_size 8 --cali_n 128 --no_grad_ckpt --split --running_stat --sm_abit 16 --cali_data_path /fastdata/users/nadavg/sd/qdiff/gen_calib/calib_dict_steps20.pt --outdir /workspace/sd/qdiff/output_quantization --ddim_steps 20 --symmetric_weight  --quant_act_ops  --accum_batches --cali_iters 20000 --cali_iters_a 5000 "]

In [30]:
sess_name = get_session_name(f'sd_{task}_wb={weight_bit}_wsym={symmetric_weight}_gpu_{gpu}')
sess_name = sess_name if not debug else sess_name + '_debug'

sess_name

'sd_quantize_wb=8_wsym=True_gpu_1'

In [31]:
tmux_session(sess_name,inst_list)

tmux attach -t "sd_quantize_wb=8_wsym=True_gpu_1"


'sd_quantize_wb=8_wsym=True_gpu_1'

In [36]:
#kill_session(kill_only=sess_name)

before kill sessions=[Session($20 sd_quantize_wb=8_wsym=True_gpu_2), Session($19 sd_quantize_wb=8_wsym=True_gpu_3)]
after kill sessions=[Session($19 sd_quantize_wb=8_wsym=True_gpu_3)]
